In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin, clone
from sklearn.preprocessing import OneHotEncoder


In [2]:
import pandas as pd

# Thử đọc tệp CSV với tùy chọn bỏ qua lỗi
df = pd.read_csv('.../Car-Evaluation/Dataset/Final/DataVersion3.csv')
df = df.drop('ad_id',axis = 1)

In [3]:
# Lưu cột 'price_in_billion' riêng biệt
price_column = df['price_in_billion']
df = df.drop(columns=['price_in_billion'])

# Chọn các cột cần áp dụng mã hóa one-hot
columns_for_encoding = ['origin', 'car_model', 'exterior_color', 'interior_color', 'engine', 'transmission', 'drive_type', 'car_name']

# Khởi tạo và fit OneHotEncoder với DataFrame
encoder = OneHotEncoder(sparse=False)
encoder.fit(df[columns_for_encoding])

# Áp dụng mã hóa one-hot cho các cột và chuyển đổi thành DataFrame mới
df_encoded = pd.DataFrame(encoder.transform(df[columns_for_encoding]), columns=encoder.get_feature_names_out(columns_for_encoding))

# Ghép cột 'price_in_billion' vào DataFrame mới
df_encoded['price_in_billion'] = price_column

# In ra DataFrame sau khi áp dụng mã hóa one-hot
df = df_encoded.dropna()

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
df.head()

,origin_Domestic assembly,origin_Imported,car_model_Convertible/Cabriolet,car_model_Coupe,car_model_Crossover,car_model_Hatchback,car_model_Pickup,car_model_SUV,car_model_Sedan,car_model_Truck,...,car_name_Volvo XC90 T6 Inscription 2016,car_name_Volvo XC90 T6 Inscription 2017,car_name_Volvo XC90 T6 Inscription 2018,car_name_Volvo XC90 T6 Inscription 2019,car_name_Volvo XC90 T6 Momentum 2016,car_name_Volvo XC90 ULTIMATE 2023,car_name_Volvo XC90 Ultimate 2022 car,car_name_Volvo XC90 Ultimate B6 2023 car,car_name_Volvo XC90 Ultimate B6 AWD 2023,price_in_billion
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.249
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.286
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.885
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.754
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.840


In [5]:
# Kiểm tra xem DataFrame có giá trị NaN hay không
has_nan = df.isnull().values.any()

print(has_nan)

False


In [6]:
# Tách dữ liệu thành đặc trưng và mục tiêu
X = df.drop(columns=['price_in_billion'])
y = df['price_in_billion']

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
# Define the model
model = RandomForestRegressor(random_state=42)

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Best parameters found: {best_params}")

# Make predictions with the best model
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

In [8]:
nan_rows = X_test.isna().any(axis=1)
X_test = X_test[~nan_rows]
y_test = y_test[~nan_rows]

In [9]:
# Dự đoán trên tập dữ liệu kiểm tra
y_pred = model.predict(X_test)

# Tính RMSE
rmse = mean_squared_error(y_test, y_pred)
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 1.5516859130970262


In [10]:

rmse = mean_squared_error(y_test, y_pred)
print(f'Root Mean Squared Error: {rmse}')

# Tính R2 score
from sklearn.metrics import mean_squared_error, r2_score
r2 = r2_score(y_test, y_pred)
print(f'R2 Score: {r2}')

Root Mean Squared Error: 1.5516859130970262
R2 Score: 0.5972206150256492
